In [1]:
!pip install "unsloth[colab-new]"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.5/61.5 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 3.0 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of torchvision to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 30.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 566.1/566.1 kB 33.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 564.7/564.7 kB 33.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 79.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 273.6/273.6 kB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.2/117.2 MB 14.6 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 888.1/888.1 MB 1.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.5/155.5 MB 2.2 MB/s eta 0:00:00:00:0100

In [2]:
%%writefile train_teacher.py
import torch
from unsloth import FastLanguageModel
from datasets import load_dataset
from trl import SFTTrainer
from transformers import TrainingArguments
import os

# --- 1. SESSION 1 CONFIG ---
TRAIN_SAMPLES = 30000
EVAL_SAMPLES = 3000
MAX_SEQ_LENGTH = 2048
SAVE_PATH = "/kaggle/working/teacher_model"

print(f"--- SESSION 1 / STAGE 1: TRAINING TEACHER MODEL ---")
print(f"Training on {TRAIN_SAMPLES} samples, validating on {EVAL_SAMPLES}.")
print(f"Using {MAX_SEQ_LENGTH} sequence length.")
print(f"Est. Time: ~10.5 Hours")

os.makedirs(SAVE_PATH, exist_ok=True)

# --- 2. LOAD MODEL & TOKENIZER ---
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/Meta-Llama-3.1-8B",
    max_seq_length=MAX_SEQ_LENGTH,
    dtype=None,
    load_in_4bit=True,
    device_map="auto",
)

model = FastLanguageModel.get_peft_model(
    model, r=16, lora_alpha=32,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    lora_dropout=0, bias="none", use_gradient_checkpointing="unsloth", random_state=42,
)

# --- 3. PREPARE DATASET ---
dataset = load_dataset("ad6398/nyu-dl-teach-maths-comp", split="train")
shuffled_dataset = dataset.shuffle(seed=42)

train_subset = shuffled_dataset.select(range(TRAIN_SAMPLES))
eval_subset = shuffled_dataset.select(range(TRAIN_SAMPLES, TRAIN_SAMPLES + EVAL_SAMPLES))

training_prompt_template = """<|begin_of_text|><|start_header_id|>system<|end_header_id|>
You are a meticulous math solution verifier. Your task is to carefully analyze the 'Provided Solution' step-by-step to determine if it logically and correctly reaches the 'Expected Answer' for the given 'Question'. Respond ONLY with 'True' if the solution's reasoning AND final result are correct and match the expected answer, otherwise respond ONLY with 'False'.<|eot_id|><|start_header_id|>user<|end_header_id|>
Question:
{}

Expected Answer:
{}

Provided Solution:
{}<|eot_id|><|start_header_id|>assistant<|end_header_id|>
{}<|eot_id|>"""

def formatting_prompts_func(examples):
    questions, solutions, answers = examples["question"], examples["solution"], examples["answer"]
    outputs = [str(x) for x in examples["is_correct"]]
    texts = []
    for question, solution, answer, output in zip(questions, solutions, answers, outputs):
        text = training_prompt_template.format(question, str(answer), str(solution), output)
        texts.append(text)
    return { "text" : texts }

try: num_proc = os.cpu_count() // 2
except: num_proc = 4

final_train_dataset = train_subset.map(formatting_prompts_func, batched=True, num_proc=num_proc)
final_eval_dataset = eval_subset.map(formatting_prompts_func, batched=True, num_proc=num_proc)

# --- 4. SET UP TRAINER ---
# Total steps = 30,000 / (2 * 8) = 1875 steps
training_args = TrainingArguments(
    output_dir="training_outputs_teacher",
    num_train_epochs=1,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    gradient_accumulation_steps=8,
    optim="adamw_8bit",
    logging_steps=100,
    learning_rate=1e-4,
    fp16=not torch.cuda.is_bf16_supported(),
    bf16=torch.cuda.is_bf16_supported(),
    seed=42,
    report_to="none",
    eval_strategy="steps",
    eval_steps=250,
    save_strategy="steps",
    save_steps=250,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False,
    save_total_limit=2,
)

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    args=training_args,
    train_dataset=final_train_dataset,
    eval_dataset=final_eval_dataset,
    dataset_text_field="text",
    max_seq_length=MAX_SEQ_LENGTH,
    packing=False,
)

# --- 5. START TRAINING ---
print("--- 🚀 Starting Teacher Model Training (30k, 2048) ---")
trainer.train()
print("--- ✅ Teacher Model Training Complete ---")

# --- 6. SAVE FINAL MODEL ---
print(f"Saving best teacher model to {SAVE_PATH}...")
trainer.model.save_pretrained(SAVE_PATH)
tokenizer.save_pretrained(SAVE_PATH)
print(f"--- ✅ Stage 1 Complete. ---")

Writing train_teacher.py


In [3]:
!python train_teacher.py

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
2025-11-02 02:01:17.806542: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1762048878.038939     116 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1762048878.102867     116 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
🦥 Unsloth Zoo will now patch everything to make training faster!
--- SESSION 1 / STAGE 1: TRAINING TEACHER MODEL ---
Training on 30000 samples, validating on 3000.
Using 2048 sequence length.
Est. Time: ~10.5 Hours
==((====))==  Unsloth 2025.10.12: Fast Llama patching. Transformers: 4.57.1.
   \\   /|    Tesla T4. Num GPUs = 2. Max memory: 14.741 GB. Platform: L